In [4]:
#extra points to add
#source, telescope, freq, flux, flux error, beam diam, date

#Herrnstein et al, 1998, APJ, 497:L69
#doesn't specify day
#these are strict brightnesses without beam considerations
#drop VLBI observations
#extra1=['NGC4258','VLBA',22,'.220*','NA',.00045,'1997/03/01']

#Doi et al, 2005, MNRAS, 363:2
extra2=['NGC4258','SCUBA-JCMT',347,93.7,29.3,15,'NA', 'Doi et al, 2005, MNRAS, 363:2']
extra3=['NGC4258','NMA',96,11,2.7,7,'NA', 'Doi et al, 2005, MNRAS, 363:2']

#Kamali, 2017, A\&A, 605:A84
#beam size for these sources do not give reliable beam sizes ranging from 0.19 to 0.50 arcsec '0.19-0.50'
extra4=['ESO558-G009','VLA',33,.80,.04,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra5=['IC2560','VLA',33, 2.00,0.10,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra6=['Mrk1029','VLA',33,1.18,0.14,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra7=['NGC1194','VLA',33,1.08,0.04,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra8=['NGC2273','VLA',33,2.69,0.29,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra9=['NGC3393','VLA',33,5.30,0.36,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra10=['NGC4388','VLA',33,8.57,0.43,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra11=['NGC5495','VLA',33,0.13,0.02,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']
extra12=['UGC3789','VLA',33,0.21,0.02,'0.19-0.50','NA','Kamali, 2017, A&A, 605:A84']

#Krips, 2006, A\&A: 446:113
#doesn't state the date '2003/02/01'
#drop VLBI observations
#extra13=['NGC1068','IRAM PdB',100,36,0.4,.21,'2003/02/01']
#extra14=['NGC1068','IRAM PdB',300,22,0.8,0.8,'2003/02/01']

extras=[]

#extras.append(extra1)

extras.append(extra2)
extras.append(extra3)

extras.append(extra4)
extras.append(extra5)
extras.append(extra6)
extras.append(extra7)
extras.append(extra8)
extras.append(extra9)
extras.append(extra10)
extras.append(extra11)
extras.append(extra12)

#extras.append(extra13)
#extras.append(extra14)

In [5]:
import ast
import re

def normalize_row(row):
    # NEW: handle list-of-1 where the sole element is a space-separated line
    if isinstance(row, list) and len(row) == 1 and isinstance(row[0], str):
        s = row[0].strip()
        # if it looks like a bracketed list, let your existing logic handle it later
        if not (s.startswith('[') and s.endswith(']')):
            # split on 2+ spaces to keep things like '0.2*' intact
            parts = [p.strip() for p in re.split(r'\s{2,}', s) if p.strip()]
            return parts

    # If row is a list of string fragments like ["'A',", "'B',", ...] -> join and eval FIRST
    if isinstance(row, list) and all(isinstance(s, str) for s in row) and any("," in s for s in row):
        try:
            row = ast.literal_eval(''.join(row))
        except Exception:
            # fall through; we'll at least clean strings
            pass

    # Now clean fields (note: NO comma stripping here)
    if isinstance(row, list):
        clean = []
        for s in row:
            if isinstance(s, str):
                s = s.strip().strip("'").strip('"').replace("\\n", "").strip()
            clean.append(s)
        return clean
    else:
        s = str(row).strip().strip("'").strip('"').replace("\\n", "").strip()
        return [s]



def date_key(s, default=-1.0):
    if s is None:
        return default
    s = str(s).strip().strip("'").strip('"').strip(',')
    if s.upper() == "NA" or s == "":
        return default
    parts = s.split("/")
    if len(parts) != 3:
        return default
    try:
        y, m, d = [float(p.strip().strip("'").strip('"').strip(',')) for p in parts]
        return y + m/12.0 + d/365.0
    except Exception:
        return default


def safe_float(s):
    # Numeric or 'NA'
    try:
        return float(s)
    except Exception:
        return -1.0


In [6]:
#sort the data summary
import numpy as np
import sys
import os
os.chdir('analysis/multi_freq_from_archive')

all_lines=[]

textin=f'fitsumfiles/sorted/fitsummary_fixtochosen_sorted.txt'
textout=f'fitsumfiles/final/fitsummary_final_withextras.txt'

if not os.path.exists('fitsumfiles/final'):
    os.makedirs('fitsumfiles/final')

write=True
#write=False

totex=0
with open(textout, 'w') as file:
    if write==True:
        original_stdout = sys.stdout
        sys.stdout = file
    all_lines=[]
    with open(textin, 'r') as infile:
        tline=[]
        for line in infile:
            line = line.split(' ')
            line2=[]
            for i in line:
                if i!='':
                    line2.append(i)
            all_lines.append(line2)
    source_order=[]
    doneindex=[]
    n = len(all_lines)
    for i in range(n):
        breaker=False
        for idoneindex in doneindex:
            if i==idoneindex:
                breaker=True
        if breaker==True:
            continue
        isource_order=[]
        isource_order.append(all_lines[i])
        for j in range(n):
            if j==1:
                continue
            if j!=i:
                if all_lines[i][0]==all_lines[j][0]:
                    #so that while iterating through i the source is not repeated
                    isource_order.append(all_lines[j])
                    doneindex.append(j)
            n2 = len(isource_order)
        #sorting by frequency

        for iex in extras:
            if iex[0] in isource_order[0][0]:
                #source(0), telescope(1), freq(2), flux(3), flux error(4), beam diam(5), date(6)
                # -> target(0), Telescope(1), freq(2), flux(3), date (index)(6), index: 'NA', beam size (")(5), snr (3)/(4), reference         
                #           0,      1,      2,      3,      6,      5,      3/4 
                if len(str(iex[3]).split('*'))==1:
                    exnew=[iex[0], iex[1], str(iex[2]), str(iex[3]), iex[6], str(iex[5]), f'{float(iex[3])/float(iex[4]):.3f}', f'{iex[7]}\n']
                else:
                    exnew=[iex[0], iex[1], str(iex[2]), str(iex[3]), iex[6], str(iex[5]), 'NA', f'{iex[7]}\n']
                totex=totex+1
                isource_order.append(exnew)



        # --- normalize then sort ---
        isource_order = [normalize_row(row) for row in isource_order]

        # (optional) peek
        #print("FIRST ROW (normalized):", isource_order[0])

        # Apply normalization and skip empty rows
        isource_order = [
            normalize_row(row)
            for row in isource_order
            if row and any(str(item).strip() for item in row)  # skip if row is empty or only blanks
        ]



        for i2source_order in isource_order:
            print(i2source_order)
            pass
        print('\n')
    if write==True:
        sys.stdout = original_stdout
if totex!=len(extras)-1:
    input('Extras are not all being read') 

In [7]:
#make machine readable table
totspacefind=0

import os
import sys

textin='fitsumfiles/final/fitsummary_final_withextras.txt'

if not os.path.exists('machinetables'):
    os.makedirs('machinetables')
textout='machinetables/fitsummary_machine.txt'

write=True
#write=False

spacings=[15, 15, 19, 16, 15, 19, 15,112]
decimals=['NA','NA',0,1,'NA',5,3]
totspace=0
for i in spacings:
    totspace=totspace+i

allbeams=[]
allsnr=[]

totex=0

with open(textout, 'w') as file:
    if write==True:
        original_stdout = sys.stdout
        sys.stdout = file
    all_lines=[]
    with open(textin, 'r') as infile:
        tline=[]
        for line in infile:
            line = line.split(' ')
            line2=[]
            for i in line:
                if i!='':
                    line2.append(i)
            all_lines.append(line2)
    source_order=[]
    doneindex=[]
    n = len(all_lines)
    for i in range(n):
        if i==0:
            continue
        breaker=False
        for idoneindex in doneindex:
            if i==idoneindex:
                breaker=True
        if breaker==True:
            continue
        isource_order=[]
        isource_order.append(all_lines[i])

        for j in range(n):
            if j==1:
                continue
            if j!=i:
                if all_lines[i][0]==all_lines[j][0]:
                    #so that while iterating through i the source is not repeated
                    isource_order.append(all_lines[j])
                    doneindex.append(j)
            n2 = len(isource_order)
        #sorting by frequency

        for iex in extras:
            if iex[0] in isource_order[0][0]:
                # source(0), telescope(1), freq(2), flux(3), flux error(4), beam diam(5), date(6)                       
                # -> target(0), Telescope(1), freq(2), flux(3), date (6), index: 'NA', beam size (")(5), snr (3)/(4)    
                #           0,      1,      2,      3,      6,     'NA',     5,      3/4                            
                if len(str(iex[3]).split('*'))==1:
                    exnew=[iex[0], iex[1], str(iex[2]), str(iex[3]), iex[6],  str(iex[5]), f'{float(iex[3])/float(iex[4]):.3f}',f'{iex[7]}']
                else:
                    exnew=[iex[0], iex[1], str(iex[2]), str(iex[3]), iex[6],  str(iex[5]), 'NA',f'{iex[7]}']
                totex=totex+1
                isource_order.append(exnew)

        isource_order = [
            normalize_row(row)
            for row in isource_order
            if row and any(str(item).strip() for item in row)  # skip empty or all-blank
        ]


        isource_order = sorted(
            isource_order,
            key=lambda x: (
                safe_float(x[2]),  # frequency
                date_key(x[4]),    # date or NA
                safe_float(x[5])   # beam or NA
            )
        )
    

        for i2source_order in isource_order:
            i2source_order = normalize_row(i2source_order)  # <-- add this
            #decimals=['NA','NA',0,2,'NA',5,3]
            #freq
            freq=float(i2source_order[2])
            if freq<10:
                new2=f'{freq:.1f}'
            else:
                new2=f'{freq:.0f}'
            #flux
            flux=i2source_order[3]
            if float(flux.split('*')[0])<0.1:
                #input('error')
                #input(flux)
                pass
            if len(flux.split('*'))>1:
                flux=float(flux.split('*')[0])
                flux=f'{flux:.{1}f}*'
            else:
                flux=float(flux)
                flux=f'{flux:.{1}f}'
            new3=flux
            beam=i2source_order[5]
            if '-'in i2source_order[5]:
                beam=i2source_order[5]
            else:
                if not  isinstance(beam, float):
                    beam=float(i2source_order[5])
                if beam<0.1:
                    beam=f'{beam:.4f}'
                elif beam<1:
                    beam=f'{beam:.3f}'
                elif beam<10:
                    beam=f'{beam:.2f}'
                elif beam<100:
                    beam=f'{beam:.1f}'
                else:
                    input(f'problem with beam {beam}')
            
            allbeams.append(beam)
            new5=beam
            #snr
            snr=i2source_order[6]
            snr=snr.replace("\n", "")
            if snr!='NA':
                allsnr.append(snr)
                snr=float(i2source_order[6])
                snr=f'{snr:.0f}'
            new6=snr
            
            new7=i2source_order[7].replace("\n", "")
            if '/' in new7:
                new7=new7.split('/')[2]
            if 'Gaussian_Fit_Pipeline' in new7:
                new7='Burridge, 2025'
            if len(new7)> totspacefind:
                totspacefind=len(new7)
                newlonval=new7
            print(f"{i2source_order[0].ljust(spacings[0])}{i2source_order[1].ljust(spacings[1])}{new2.ljust(spacings[2])}{new3.ljust(spacings[3])}{i2source_order[4].ljust(spacings[4])}{new5.ljust(spacings[5])}{new6.ljust(spacings[6])}{new7.ljust(spacings[7])}")
    

    if write==True:
        sys.stdout = original_stdout


In [8]:
#check the length of the longest file name
print(totspacefind)
print(newlonval)

In [ ]:
#run scripts to get graphs, contamination
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

with open("Beam_Date_Flux_Graphs.ipynb") as f:
    nb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(nb, {"metadata": {"path": "./"}})

with open("Variability_Search.ipynb") as f:
    nb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(nb, {"metadata": {"path": "./"}})

with open("flux_size_graph.ipynb") as f:
    nb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(nb, {"metadata": {"path": "./"}})

with open("get_contams.ipynb") as f:
    nb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(nb, {"metadata": {"path": "./"}})